In [1]:
import numpy as np
from tqdm.auto import tqdm

# import testDatasetUtils as tdu

import warnings
warnings.filterwarnings("ignore")
import constan as constant
import predeal
import tools
import DT as dt

In [2]:
import os


IFPrint = False
max_iter=10

def write_acc(fileName,dataSetName,acc):

    accWrite = []
    # 判断acc中是否包含某个key
    if 'myTree_mc' in acc.keys() and len(acc['myTree_mc']) != 0:
        accWrite.append(format(np.mean(acc['myTree_mc']),'.3f'))
    if 'T_median' in acc.keys() and len(acc['T_median']) != 0:
        accWrite.append(format(np.mean(acc['T_median']),'.3f'))
    if 'T_mean' in acc.keys() and len(acc['T_mean']) != 0:
        accWrite.append(format(np.mean(acc['T_mean']),'.3f'))
    if 'standard' in acc.keys() and len(acc['standard']) != 0:
        accWrite.append(format(np.mean(acc['standard']),'.3f'))
    if 'nearestCentroid' in acc.keys() and len(acc['nearestCentroid']) != 0:
        accWrite.append(format(np.mean(acc['nearestCentroid']),'.3f'))
    if 'myTree_mc_rank' in acc.keys() and len(acc['myTree_mc_rank']) != 0:
        accWrite.append(format(np.mean(acc['myTree_mc_rank']),'.3f'))
    if 'T_median_rank' in acc.keys() and len(acc['T_median_rank']) != 0:
        accWrite.append(format(np.mean(acc['T_median_rank']),'.3f'))
    if 'myTree_mc_edit' in acc.keys() and len(acc['myTree_mc_edit']) != 0:
        accWrite.append(format(np.mean(acc['myTree_mc_edit']),'.3f'))
    if 'T_median_edit' in acc.keys() and len(acc['T_median_edit']) != 0:
        accWrite.append(format(np.mean(acc['T_median_edit']),'.3f'))


    # 如果文件中还没有数据，就写入数据，如果有数据，向后追加数据
    if not os.path.exists(fileName):
        with open(fileName, 'w') as f:
            f.write('%s,'%(dataSetName))
            f.write(','.join(accWrite))
    else:
        with open(fileName, 'a+') as f:
            f.write('%s,'%(dataSetName))
            f.write(','.join(accWrite))
            
    #向文件中添加空行
    with open(fileName, 'a+') as f:
        f.write('\n')

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle


def contrastExperiment_numerical(X, y, curDepth=0, maxLeafSize=1, meanWay=None, maxDepth=1000000000):
        k = 5
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=None)
        acc = {'myTree_mc':[],'T_median':[],'T_mean':[],'standard': [], 'nearestCentroid': []}
        X_normalized = predeal.normalization(X)
        for train_index, test_index in skf.split(X_normalized, y):
            X_train_normalized, X_test_normalized = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            X_train_normalized, y_train = shuffle(X_train_normalized, y_train)

            constant.set_value('gl_Xtrain', X_train_normalized)
            constant.set_value('gl_ytrain', y_train)
            constant.set_value('gl_distances', tools.calcDistancesMetric('numerical','euclidean',X_train_normalized))

            T_gini_mc = dt.DT()
            indeices = [index for index in range(len(X_train_normalized))]
            T_gini_mc.fit(indeices)
            acc['myTree_mc'].append(T_gini_mc.score(X_test_normalized, y_test))

            
        print(acc)
        return acc

In [4]:
def test_iris(curDepth=0, maxLeafSize=1, meanWay='MEDIAN', maxDepth=1000000000):
    constant._init()
    from sklearn.datasets import load_iris
    iris = load_iris()
    X = iris.data
    y = iris.target
    # print_data_info(dataType='numerical',fileName='iris', X=X, y=y)
    return contrastExperiment_numerical(X, y, curDepth, maxLeafSize, meanWay, maxDepth)

In [5]:
test_iris()

TypeError: _vhstack_dispatcher() takes 1 positional argument but 2 were given